In [4]:
import json
import os

import importlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import seaborn as sns
from sklearn import metrics
import torch

from models import avg_speed, schedule, ff, persistent, rnn, transformer, conv
from utils import data_utils, data_loader, model_utils, shape_utils

importlib.reload(data_utils)
importlib.reload(data_loader)
importlib.reload(shape_utils)

%matplotlib inline

run_folder = "../results/medium/"
run_folder_tte = "../results/small/"
FOLD_MODEL = 0
device = torch.device("cpu")
NUM_WORKERS = 4
HIDDEN_SIZE = 32
BATCH_SIZE = 64

# Set hyperparameters
EPOCH_EVAL_FREQ = 2

# Define embedded variables for network models
embed_dict = {
    'timeID': {
        'vocab_size': 1440,
        'embed_dims': 24
    },
    'weekID': {
        'vocab_size': 7,
        'embed_dims': 4
    }
}

# Get list of available train/test files
kcm_data_folder = f"{run_folder}kcm/deeptte_formatted/"
atb_data_folder = f"{run_folder}atb/deeptte_formatted/"

train_file_list_kcm = list(filter(lambda x: x[:5]=="train" and len(x)==6, os.listdir(kcm_data_folder)))
test_file_list_kcm = list(filter(lambda x: x[:4]=="test" and len(x)==5, os.listdir(kcm_data_folder)))
train_file_list_atb = list(filter(lambda x: x[:5]=="train" and len(x)==6, os.listdir(atb_data_folder)))
test_file_list_atb = list(filter(lambda x: x[:4]=="test" and len(x)==5, os.listdir(atb_data_folder)))

In [ ]:
# Load KCM Models

In [5]:
# Declare baseline models
avg_model = data_utils.load_pkl(f"{run_folder}kcm/models/AVG_{FOLD_MODEL}.pkl")
sch_model = data_utils.load_pkl(f"{run_folder}kcm/models/SCH_{FOLD_MODEL}.pkl")
tim_model = data_utils.load_pkl(f"{run_folder}kcm/models/PER_TIM_{FOLD_MODEL}.pkl")

# Declare network models
ff_model = ff.FF(
    "FF",
    11,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
ff_model.load_state_dict(torch.load(f"{run_folder}kcm/models/{ff_model.model_name}_{FOLD_MODEL}.pt"))
print(ff_model.model_name)
print(sum(p.numel() for p in ff_model.parameters() if p.requires_grad))
gru_model = rnn.GRU_RNN(
    "GRU_RNN",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
gru_model.load_state_dict(torch.load(f"{run_folder}kcm/models/{gru_model.model_name}_{FOLD_MODEL}.pt"))
print(gru_model.model_name)
print(sum(p.numel() for p in gru_model.parameters() if p.requires_grad))
gru_mto_model = rnn.GRU_RNN_MTO(
    "GRU_RNN_MTO",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
gru_mto_model.load_state_dict(torch.load(f"{run_folder}kcm/models/{gru_mto_model.model_name}_{FOLD_MODEL}.pt"))
print(gru_mto_model.model_name)
print(sum(p.numel() for p in gru_mto_model.parameters() if p.requires_grad))
conv1d_model = conv.CONV(
    "CONV1D",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
conv1d_model.load_state_dict(torch.load(f"{run_folder}kcm/models/{conv1d_model.model_name}_{FOLD_MODEL}.pt"))
print(conv1d_model.model_name)
print(sum(p.numel() for p in conv1d_model.parameters() if p.requires_grad))
trs_model = transformer.TRANSFORMER(
    "TRSF_ENC",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
trs_model.load_state_dict(torch.load(f"{run_folder}kcm/models/{trs_model.model_name}_{FOLD_MODEL}.pt"))
print(trs_model.model_name)
print(sum(p.numel() for p in trs_model.parameters() if p.requires_grad))

# Add all models to results list
model_list = []
model_list.append(avg_model)
model_list.append(sch_model)
model_list.append(tim_model)
model_list.append(ff_model)
model_list.append(gru_model)
model_list.append(gru_mto_model)
model_list.append(conv1d_model)
model_list.append(trs_model)

FF
36957
GRU_RNN
38681
GRU_RNN_MTO
38681
CONV1D
35449
TRSF_ENC
50313


In [7]:
print(f"Evaluating KCM on KCM")
model_fold_results = {}
for x in model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}
for valid_file in test_file_list_kcm:
    print(f"VALIDATE ON FILE: {valid_file}")

    # These are fold holdouts, separate validation files are used for generalization
    valid_data = data_utils.load_all_data(kcm_data_folder, valid_file)
    with open(f"{kcm_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for network models
    valid_dataloader_basic = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS)
    valid_dataloader_seq = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS)
    valid_dataloader_seq_mto = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS)
    valid_dataloader_trs = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS)
    print(f"Successfully loaded {len(valid_data)} testing samples.")

    print(f"Evaluating {avg_model.model_name}")
    avg_labels, avg_preds = avg_model.predict(valid_dataloader_basic, config)
    model_fold_results[avg_model.model_name]["Labels"].extend(list(avg_labels))
    model_fold_results[avg_model.model_name]["Preds"].extend(list(avg_preds))

    print(f"Evaluating {sch_model.model_name}")
    sch_labels, sch_preds = sch_model.predict(valid_dataloader_basic, config)
    model_fold_results[sch_model.model_name]["Labels"].extend(list(sch_labels))
    model_fold_results[sch_model.model_name]["Preds"].extend(list(sch_preds))

    print(f"Evaluating {tim_model.model_name}")
    tim_labels, tim_preds = tim_model.predict(valid_dataloader_seq, config)
    model_fold_results[tim_model.model_name]["Labels"].extend(list(tim_labels))
    model_fold_results[tim_model.model_name]["Preds"].extend(list(tim_preds))

    print(f"Evaluating {ff_model.model_name}")
    ff_labels, ff_preds = ff_model.evaluate(valid_dataloader_basic, config)
    model_fold_results[ff_model.model_name]["Labels"].extend(list(ff_labels))
    model_fold_results[ff_model.model_name]["Preds"].extend(list(ff_preds))

    print(f"Evaluating {gru_model.model_name}")
    gru_labels, gru_preds = gru_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[gru_model.model_name]["Labels"].extend(list(gru_labels))
    model_fold_results[gru_model.model_name]["Preds"].extend(list(gru_preds))

    print(f"Evaluating {gru_mto_model.model_name}")
    gru_mto_labels, gru_mto_preds = gru_mto_model.evaluate(valid_dataloader_seq_mto, config)
    model_fold_results[gru_mto_model.model_name]["Labels"].extend(list(gru_mto_labels))
    model_fold_results[gru_mto_model.model_name]["Preds"].extend(list(gru_mto_preds))

    print(f"Evaluating {conv1d_model.model_name}")
    conv1d_labels, conv1d_preds = conv1d_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[conv1d_model.model_name]["Labels"].extend(list(conv1d_labels))
    model_fold_results[conv1d_model.model_name]["Preds"].extend(list(conv1d_preds))

    print(f"Evaluating {trs_model.model_name}")
    trs_labels, trs_preds = trs_model.evaluate(valid_dataloader_trs, config)
    model_fold_results[trs_model.model_name]["Labels"].extend(list(trs_labels))
    model_fold_results[trs_model.model_name]["Preds"].extend(list(trs_preds))
kcm_kcm_model_fold_results = model_fold_results

print(f"Evaluating KCM on ATB")
model_fold_results = {}
for x in model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}
for valid_file in test_file_list_atb:
    print(f"VALIDATE ON FILE: {valid_file}")

    # These are fold holdouts, separate validation files are used for generalization
    valid_data = data_utils.load_all_data(atb_data_folder, valid_file)
    with open(f"{kcm_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for network models
    valid_dataloader_basic = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS)
    valid_dataloader_seq = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS)
    valid_dataloader_seq_mto = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS)
    valid_dataloader_trs = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS)
    print(f"Successfully loaded {len(valid_data)} testing samples.")

    print(f"Evaluating {avg_model.model_name}")
    avg_labels, avg_preds = avg_model.predict(valid_dataloader_basic, config)
    model_fold_results[avg_model.model_name]["Labels"].extend(list(avg_labels))
    model_fold_results[avg_model.model_name]["Preds"].extend(list(avg_preds))

    print(f"Evaluating {sch_model.model_name}")
    sch_labels, sch_preds = sch_model.predict(valid_dataloader_basic, config)
    model_fold_results[sch_model.model_name]["Labels"].extend(list(sch_labels))
    model_fold_results[sch_model.model_name]["Preds"].extend(list(sch_preds))

    print(f"Evaluating {tim_model.model_name}")
    tim_labels, tim_preds = tim_model.predict(valid_dataloader_seq, config)
    model_fold_results[tim_model.model_name]["Labels"].extend(list(tim_labels))
    model_fold_results[tim_model.model_name]["Preds"].extend(list(tim_preds))

    print(f"Evaluating {ff_model.model_name}")
    ff_labels, ff_preds = ff_model.evaluate(valid_dataloader_basic, config)
    model_fold_results[ff_model.model_name]["Labels"].extend(list(ff_labels))
    model_fold_results[ff_model.model_name]["Preds"].extend(list(ff_preds))

    print(f"Evaluating {gru_model.model_name}")
    gru_labels, gru_preds = gru_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[gru_model.model_name]["Labels"].extend(list(gru_labels))
    model_fold_results[gru_model.model_name]["Preds"].extend(list(gru_preds))

    print(f"Evaluating {gru_mto_model.model_name}")
    gru_mto_labels, gru_mto_preds = gru_mto_model.evaluate(valid_dataloader_seq_mto, config)
    model_fold_results[gru_mto_model.model_name]["Labels"].extend(list(gru_mto_labels))
    model_fold_results[gru_mto_model.model_name]["Preds"].extend(list(gru_mto_preds))

    print(f"Evaluating {conv1d_model.model_name}")
    conv1d_labels, conv1d_preds = conv1d_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[conv1d_model.model_name]["Labels"].extend(list(conv1d_labels))
    model_fold_results[conv1d_model.model_name]["Preds"].extend(list(conv1d_preds))

    print(f"Evaluating {trs_model.model_name}")
    trs_labels, trs_preds = trs_model.evaluate(valid_dataloader_trs, config)
    model_fold_results[trs_model.model_name]["Labels"].extend(list(trs_labels))
    model_fold_results[trs_model.model_name]["Preds"].extend(list(trs_preds))
kcm_atb_model_fold_results = model_fold_results


Evaluating KCM on KCM
VALIDATE ON FILE: test3
Successfully loaded 12979 testing samples.
Evaluating AVG
Evaluating SCH
Evaluating PER_TIM
Evaluating FF
Evaluating GRU_RNN
Evaluating GRU_RNN_MTO
Evaluating CONV1D
Evaluating TRSF_ENC
VALIDATE ON FILE: test1
Successfully loaded 40600 testing samples.
Evaluating AVG
Evaluating SCH
Evaluating PER_TIM
Evaluating FF
Evaluating GRU_RNN
Evaluating GRU_RNN_MTO
Evaluating CONV1D
Evaluating TRSF_ENC
VALIDATE ON FILE: test0
Successfully loaded 41912 testing samples.
Evaluating AVG
Evaluating SCH
Evaluating PER_TIM
Evaluating FF
Evaluating GRU_RNN
Evaluating GRU_RNN_MTO
Evaluating CONV1D
Evaluating TRSF_ENC
VALIDATE ON FILE: test2
Successfully loaded 13522 testing samples.
Evaluating AVG
Evaluating SCH
Evaluating PER_TIM
Evaluating FF
Evaluating GRU_RNN
Evaluating GRU_RNN_MTO
Evaluating CONV1D
Evaluating TRSF_ENC
VALIDATE ON FILE: test4
Successfully loaded 20084 testing samples.
Evaluating AVG
Evaluating SCH
Evaluating PER_TIM
Evaluating FF
Evaluati

/home/zack/miniconda3/envs/valle_m1/lib/python3.9/site-packages/torch/nn/modules/loss.py:988: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


RuntimeError: zero-dimensional tensor (at position 19) cannot be concatenated

In [ ]:
# Load ATB Models

In [ ]:
# Declare baseline models
avg_model = data_utils.load_pkl(f"{run_folder}atb/models/AVG_{FOLD_MODEL}.pkl")
sch_model = data_utils.load_pkl(f"{run_folder}atb/models/SCH_{FOLD_MODEL}.pkl")
tim_model = data_utils.load_pkl(f"{run_folder}atb/models/PER_TIM_{FOLD_MODEL}.pkl")

# Declare network models
ff_model = ff.FF(
    "FF",
    11,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
ff_model.load_state_dict(torch.load(f"{run_folder}atb/models/{ff_model.model_name}_{FOLD_MODEL}.pt"))
print(ff_model.model_name)
print(sum(p.numel() for p in ff_model.parameters() if p.requires_grad))
gru_model = rnn.GRU_RNN(
    "GRU_RNN",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
gru_model.load_state_dict(torch.load(f"{run_folder}atb/models/{gru_model.model_name}_{FOLD_MODEL}.pt"))
print(gru_model.model_name)
print(sum(p.numel() for p in gru_model.parameters() if p.requires_grad))
gru_mto_model = rnn.GRU_RNN_MTO(
    "GRU_RNN_MTO",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
gru_mto_model.load_state_dict(torch.load(f"{run_folder}atb/models/{gru_mto_model.model_name}_{FOLD_MODEL}.pt"))
print(gru_mto_model.model_name)
print(sum(p.numel() for p in gru_mto_model.parameters() if p.requires_grad))
conv1d_model = conv.CONV(
    "CONV1D",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
conv1d_model.load_state_dict(torch.load(f"{run_folder}atb/models/{conv1d_model.model_name}_{FOLD_MODEL}.pt"))
print(conv1d_model.model_name)
print(sum(p.numel() for p in conv1d_model.parameters() if p.requires_grad))
trs_model = transformer.TRANSFORMER(
    "TRSF_ENC",
    8,
    1,
    HIDDEN_SIZE,
    BATCH_SIZE,
    embed_dict,
    device
).to(device)
trs_model.load_state_dict(torch.load(f"{run_folder}atb/models/{trs_model.model_name}_{FOLD_MODEL}.pt"))
print(trs_model.model_name)
print(sum(p.numel() for p in trs_model.parameters() if p.requires_grad))

# Add all models to results list
model_list = []
model_list.append(avg_model)
model_list.append(sch_model)
model_list.append(tim_model)
model_list.append(ff_model)
model_list.append(gru_model)
model_list.append(gru_mto_model)
model_list.append(conv1d_model)
model_list.append(trs_model)

In [ ]:
print(f"Evaluating ATB on ATB")
model_fold_results = {}
for x in model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}
for valid_file in test_file_list_atb:
    print(f"VALIDATE ON FILE: {valid_file}")

    # These are fold holdouts, separate validation files are used for generalization
    valid_data = data_utils.load_all_data(atb_data_folder, valid_file)
    with open(f"{atb_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for network models
    valid_dataloader_basic = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS)
    valid_dataloader_seq = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS)
    valid_dataloader_seq_mto = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS)
    valid_dataloader_trs = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS)
    print(f"Successfully loaded {len(valid_data)} testing samples.")

    print(f"Evaluating {avg_model.model_name}")
    avg_labels, avg_preds = avg_model.predict(valid_dataloader_basic, config)
    model_fold_results[avg_model.model_name]["Labels"].extend(list(avg_labels))
    model_fold_results[avg_model.model_name]["Preds"].extend(list(avg_preds))

    print(f"Evaluating {sch_model.model_name}")
    sch_labels, sch_preds = sch_model.predict(valid_dataloader_basic, config)
    model_fold_results[sch_model.model_name]["Labels"].extend(list(sch_labels))
    model_fold_results[sch_model.model_name]["Preds"].extend(list(sch_preds))

    print(f"Evaluating {tim_model.model_name}")
    tim_labels, tim_preds = tim_model.predict(valid_dataloader_seq, config)
    model_fold_results[tim_model.model_name]["Labels"].extend(list(tim_labels))
    model_fold_results[tim_model.model_name]["Preds"].extend(list(tim_preds))

    print(f"Evaluating {ff_model.model_name}")
    ff_labels, ff_preds = ff_model.evaluate(valid_dataloader_basic, config)
    model_fold_results[ff_model.model_name]["Labels"].extend(list(ff_labels))
    model_fold_results[ff_model.model_name]["Preds"].extend(list(ff_preds))

    print(f"Evaluating {gru_model.model_name}")
    gru_labels, gru_preds = gru_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[gru_model.model_name]["Labels"].extend(list(gru_labels))
    model_fold_results[gru_model.model_name]["Preds"].extend(list(gru_preds))

    print(f"Evaluating {gru_mto_model.model_name}")
    gru_mto_labels, gru_mto_preds = gru_mto_model.evaluate(valid_dataloader_seq_mto, config)
    model_fold_results[gru_mto_model.model_name]["Labels"].extend(list(gru_mto_labels))
    model_fold_results[gru_mto_model.model_name]["Preds"].extend(list(gru_mto_preds))

    print(f"Evaluating {conv1d_model.model_name}")
    conv1d_labels, conv1d_preds = conv1d_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[conv1d_model.model_name]["Labels"].extend(list(conv1d_labels))
    model_fold_results[conv1d_model.model_name]["Preds"].extend(list(conv1d_preds))

    print(f"Evaluating {trs_model.model_name}")
    trs_labels, trs_preds = trs_model.evaluate(valid_dataloader_trs, config)
    model_fold_results[trs_model.model_name]["Labels"].extend(list(trs_labels))
    model_fold_results[trs_model.model_name]["Preds"].extend(list(trs_preds))
atb_atb_model_fold_results = model_fold_results

print(f"Evaluating ATB on KCM")
model_fold_results = {}
for x in model_list:
    model_fold_results[x.model_name] = {"Labels":[], "Preds":[]}
for valid_file in test_file_list_kcm:
    print(f"VALIDATE ON FILE: {valid_file}")

    # These are fold holdouts, separate validation files are used for generalization
    valid_data = data_utils.load_all_data(kcm_data_folder, valid_file)
    with open(f"{kcm_data_folder}train_config.json", "r") as f:
        config = json.load(f)

    # Construct dataloaders for network models
    valid_dataloader_basic = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.basic_collate, NUM_WORKERS)
    valid_dataloader_seq = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_collate, NUM_WORKERS)
    valid_dataloader_seq_mto = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.sequential_mto_collate, NUM_WORKERS)
    valid_dataloader_trs = data_loader.make_generic_dataloader(valid_data, config, BATCH_SIZE, data_loader.transformer_collate, NUM_WORKERS)
    print(f"Successfully loaded {len(valid_data)} testing samples.")

    print(f"Evaluating {avg_model.model_name}")
    avg_labels, avg_preds = avg_model.predict(valid_dataloader_basic, config)
    model_fold_results[avg_model.model_name]["Labels"].extend(list(avg_labels))
    model_fold_results[avg_model.model_name]["Preds"].extend(list(avg_preds))

    print(f"Evaluating {sch_model.model_name}")
    sch_labels, sch_preds = sch_model.predict(valid_dataloader_basic, config)
    model_fold_results[sch_model.model_name]["Labels"].extend(list(sch_labels))
    model_fold_results[sch_model.model_name]["Preds"].extend(list(sch_preds))

    print(f"Evaluating {tim_model.model_name}")
    tim_labels, tim_preds = tim_model.predict(valid_dataloader_seq, config)
    model_fold_results[tim_model.model_name]["Labels"].extend(list(tim_labels))
    model_fold_results[tim_model.model_name]["Preds"].extend(list(tim_preds))

    print(f"Evaluating {ff_model.model_name}")
    ff_labels, ff_preds = ff_model.evaluate(valid_dataloader_basic, config)
    model_fold_results[ff_model.model_name]["Labels"].extend(list(ff_labels))
    model_fold_results[ff_model.model_name]["Preds"].extend(list(ff_preds))

    print(f"Evaluating {gru_model.model_name}")
    gru_labels, gru_preds = gru_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[gru_model.model_name]["Labels"].extend(list(gru_labels))
    model_fold_results[gru_model.model_name]["Preds"].extend(list(gru_preds))

    print(f"Evaluating {gru_mto_model.model_name}")
    gru_mto_labels, gru_mto_preds = gru_mto_model.evaluate(valid_dataloader_seq_mto, config)
    model_fold_results[gru_mto_model.model_name]["Labels"].extend(list(gru_mto_labels))
    model_fold_results[gru_mto_model.model_name]["Preds"].extend(list(gru_mto_preds))

    print(f"Evaluating {conv1d_model.model_name}")
    conv1d_labels, conv1d_preds = conv1d_model.evaluate(valid_dataloader_seq, config)
    model_fold_results[conv1d_model.model_name]["Labels"].extend(list(conv1d_labels))
    model_fold_results[conv1d_model.model_name]["Preds"].extend(list(conv1d_preds))

    print(f"Evaluating {trs_model.model_name}")
    trs_labels, trs_preds = trs_model.evaluate(valid_dataloader_trs, config)
    model_fold_results[trs_model.model_name]["Labels"].extend(list(trs_labels))
    model_fold_results[trs_model.model_name]["Preds"].extend(list(trs_preds))
atb_kcm_model_fold_results = model_fold_results


In [ ]:
models = []
train_nets = []
test_nets = []
errors = []
results = kcm_kcm_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("KCM")
    test_nets.append("KCM")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = kcm_atb_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("KCM")
    test_nets.append("ATB")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = atb_atb_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("ATB")
    test_nets.append("ATB")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))
results = atb_kcm_model_fold_results
for model_name in results.keys():
    models.append(model_name)
    train_nets.append("ATB")
    test_nets.append("KCM")
    errors.append(np.round(np.sqrt(metrics.mean_squared_error(results[model_name]['Labels'], results[model_name]['Preds'])), 2))

metric_results = pd.DataFrame(columns=["Model","Train Network","Test Network","Metric"])
metric_results['Model'] = models
metric_results['Train Network'] = train_nets
metric_results['Test Network'] = test_nets
metric_results['Metric'] = errors
metric_results['Model-Train-Test'] = metric_results['Model']+"_"+metric_results['Train Network']+"_"+metric_results['Test Network']
metric_results = metric_results.sort_values('Model-Train-Test')
metric_results

In [ ]:
# Include DeepTTE Generalization Results
kcm_deeptte_gen_res = data_utils.extract_deeptte_results("KCM", run_folder_tte, "kcm/", generalization_flag=True)
atb_deeptte_gen_res = data_utils.extract_deeptte_results("ATB", run_folder_tte, "atb/", generalization_flag=True)
deeptte_gen_res = pd.concat([kcm_deeptte_gen_res, atb_deeptte_gen_res])
deeptte_gen_res["Model"] = "DEEPTTE"
deeptte_gen_res["Train Network"] = deeptte_gen_res["City"]
deeptte_gen_res["Test Network"] = deeptte_gen_res["Loss Set"]
deeptte_gen_res["Metric"] = deeptte_gen_res["RMSE"]
deeptte_gen_res = deeptte_gen_res[["Model","Train Network","Test Network","Metric"]]
deeptte_gen_res['Model-Train-Test'] = deeptte_gen_res['Model']+"_"+deeptte_gen_res['Train Network']+"_"+deeptte_gen_res['Test Network']
metric_results = pd.concat([metric_results, deeptte_gen_res])
metric_results = metric_results.sort_values('Model-Train-Test')
metric_results

In [ ]:
plot_data = metric_results[metric_results['Test Network']=="KCM"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(6)
sns.barplot(plot_data, x="Metric", y="Model-Train-Test", hue="Model", dodge=False)
axes.set_ylabel("Model")
axes.set_xlabel("RMSE")
fig.suptitle('Model Generalization Performance On KCM', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = metric_results[metric_results['Test Network']=="ATB"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(6)
sns.barplot(plot_data, x="Metric", y="Model-Train-Test", hue="Model", dodge=False)
axes.set_ylabel("Model")
axes.set_xlabel("RMSE")
fig.suptitle('Model Generalization Performance On ATB', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization.png", dpi=1800, bbox_inches='tight')